# Some example code for creating a db

In [ ]:
import sqlite3
import pandas as pd
import numpy as np

# Example: your DataFrame 'df' already exists

# Mapping function to preserve pandas dtypes in SQLite schema
def map_dtype(dtype):
    if np.issubdtype(dtype, np.datetime64):
        return "TIMESTAMP"
    elif np.issubdtype(dtype, np.bool_):
        return "BOOLEAN"
    elif np.issubdtype(dtype, np.integer):
        return "INTEGER"
    elif np.issubdtype(dtype, np.floating):
        return "REAL"
    else:
        return "TEXT"

# Define explicit schema
schema = ",\n  ".join(f"{col} {map_dtype(dtype)}" for col, dtype in df.dtypes.items())
create_table = f"CREATE TABLE IF NOT EXISTS trades (\n  {schema}\n);"

# Create database and table
conn = sqlite3.connect("openinsider.db")
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS trades;")
cur.execute(create_table)

# Insert data using parameterized executemany to avoid dtype inference
insert_sql = f"INSERT INTO trades ({', '.join(df.columns)}) VALUES ({', '.join(['?'] * len(df.columns))});"

# Convert datetime and bool to compatible SQLite types explicitly
df_to_insert = df.copy()
for c in df_to_insert.select_dtypes(include="datetime"):
    df_to_insert[c] = df_to_insert[c].astype(str)
for c in df_to_insert.select_dtypes(include="bool"):
    df_to_insert[c] = df_to_insert[c].astype(int)

cur.executemany(insert_sql, df_to_insert.itertuples(index=False, name=None))
conn.commit()

# Fetch back data with enforced dtypes
dtypes_dict = df.dtypes.to_dict()
df_loaded = pd.read_sql_query("SELECT * FROM trades;", conn)
for c, dtype in dtypes_dict.items():
    if np.issubdtype(dtype, np.datetime64):
        df_loaded[c] = pd.to_datetime(df_loaded[c])
    elif np.issubdtype(dtype, np.bool_):
        df_loaded[c] = df_loaded[c].astype(bool)
    else:
        df_loaded[c] = df_loaded[c].astype(dtype)

conn.close()
